softmax is used for multiclass classification while sigmoid is used for binary classification.when  y hat is multiple classes the last layer of neural network should have softmax activation


the point of cbow is to create vector representations so that we can come to conclusions like:
king-man+woman=queen

vocabulary is the set of all unique words inside a corpus

one hot encoding of a word is a vector of size of vocab set to 0 except the single words index in the vocabulary set to 1

we build a neural network that given a set of contex words (which acc to markov property neednt be more than 3)
the output should be a multiclass prediction vector of the size of the vocabulary of the corpus 
containing probability of the next word being the ith word in that vector


so input could be one hot encoding of 2 neuron i and am
output vector should have argmax of standing

this is the fake problem which while being trained we also get
the actual vector representations of words

categorical crossentropy is the loss function used for multi class calssficiation problems


so our vector representation exists for every word in the vocabulary but the size of this vector is completely our choice as it is the size of the second layer


king     + woman  ==?

[         [

0.9       0.1
0.2       0.2
0.1       0.3
0.2       0.8
]         ]


this is the use of vector embeddings


it is nothing but the weights corresponding to the nth network
o
o
o
o  V                       O   V
                           O
    V*W      O       V*W   O
             O             O
             O   W
o
o
o
o  V



INPUT neurons number Is the window size 


In [177]:
import numpy as np 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding,Lambda
import keras.backend as K
import gensim
from keras.preprocessing import sequence

from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer


In [178]:
with open("corona.txt",'w') as file:
    file.write("""
speed of transmission is an important point of difference between the two viruses.
Influenza has a shorter median incubation period (the time from infection to appearance of symptoms)
and a shorter serial interval (the time between successive cases) than COVID-19 virus.
The serial interval for COVID-19 virus is estimated to be 5-6 days, while for influenza virus, the serial interval is 3 days.
This means that influenza can spread faster than COVID-19.

Further, transmission in the first 3-5 days of illness, or potentially pre-symptomatic transmission –
transmission of the virus before the appearance of symptoms – is a major driver of transmission for influenza.
In contrast, while we are learning that there are people who can shed COVID-19 virus 24-48 hours prior to symptom onset,
at present, this does not appear to be a major driver of transmission.

The reproductive number – the number of secondary infections generated from one infected individual –
is understood to be between 2 and 2.5 for COVID-19 virus, higher than for influenza.
However, estimates for both COVID-19 and influenza viruses are very context and time-specific, making direct comparisons more difficult.
""")


In [179]:
data= open('corona.txt','r') 



newdata = [text for text in data if text.count(' ')>=2 ]
#as data has newline newline wise sentences
print(newdata)

# Create a tokenizer
vectorize = Tokenizer()

# Fit the tokenizer on the corpus
vectorize.fit_on_texts(newdata)

# The tokenizer now has a word index
word_index = vectorize.word_index
print(word_index.items())


['speed of transmission is an important point of difference between the two viruses.\n', 'Influenza has a shorter median incubation period (the time from infection to appearance of symptoms)\n', 'and a shorter serial interval (the time between successive cases) than COVID-19 virus.\n', 'The serial interval for COVID-19 virus is estimated to be 5-6 days, while for influenza virus, the serial interval is 3 days.\n', 'This means that influenza can spread faster than COVID-19.\n', 'Further, transmission in the first 3-5 days of illness, or potentially pre-symptomatic transmission –\n', 'transmission of the virus before the appearance of symptoms – is a major driver of transmission for influenza.\n', 'In contrast, while we are learning that there are people who can shed COVID-19 virus 24-48 hours prior to symptom onset,\n', 'at present, this does not appear to be a major driver of transmission.\n', 'The reproductive number – the number of secondary infections generated from one infected ind

In [180]:
tokens=vectorize.texts_to_sequences(newdata)
print(tokens)

#the data we have is converted into numbers

[[38, 2, 3, 9, 39, 40, 41, 2, 42, 14, 1, 43, 23], [4, 44, 11, 24, 45, 46, 47, 1, 15, 25, 48, 10, 26, 2, 27], [12, 11, 24, 16, 17, 1, 15, 14, 49, 50, 18, 5, 6, 7], [1, 16, 17, 8, 5, 6, 7, 9, 51, 10, 19, 20, 52, 21, 28, 8, 4, 7, 1, 16, 17, 9, 29, 21], [30, 53, 31, 4, 32, 54, 55, 18, 5, 6], [56, 3, 33, 1, 57, 29, 20, 21, 2, 58, 59, 60, 61, 62, 3, 13], [3, 2, 1, 7, 63, 1, 26, 2, 27, 13, 9, 11, 34, 35, 2, 3, 8, 4], [33, 64, 28, 65, 22, 66, 31, 67, 22, 68, 69, 32, 70, 5, 6, 7, 71, 72, 73, 74, 10, 75, 76], [77, 78, 30, 79, 80, 81, 10, 19, 11, 34, 35, 2, 3], [1, 82, 36, 13, 1, 36, 2, 83, 84, 85, 25, 86, 87, 88, 13], [9, 89, 10, 19, 14, 37, 12, 37, 20, 8, 5, 6, 7, 90, 18, 8, 4], [91, 92, 8, 93, 5, 6, 12, 4, 23, 22, 94, 95, 12, 15, 96, 97, 98, 99, 100, 101]]


In [181]:
word_count=sum(len(s) for s in tokens)
window_size=2
total_length=window_size*2
print(word_count)

198


In [182]:




def cbow_model(tokens,window_size,word_count):

    #tokens=[['w1','w2','w3'],['w4','w5','w6'],['w7','w8','w9']]
    #it is actually
    ##tokens=[[7,34,51],[6,8,9],[15,19,91']]
    for text in tokens:
        #text=['w1','w2','w3']
        #[7,34,51]

        text_length=len(text)
        text_length=3
        for index,word in enumerate(text):
            
            target=[]

            #index=0
            #word='w1'
            

            begin=index-window_size
            end=index+window_size+1

            context=[text[i] for i in range(begin,end) if 0<=i<=text_length and i!=index]

            #context=['w2','w3']  in numeric 
            #context=[34,51]

            padded_array = context + [0] * max(0, window_size*2 - len(context))
            padded_array = padded_array[:window_size*2]
            # contexts=sequence.pad_sequences(context,maxlen=window_size*2)
            #pad sequences converts the context array into fix size,here it is size 4




            target.append(word)
            #target=['w1'] in numeric
            #target=[7]

            categoricaltarget=to_categorical(target,word_count)
            #['w1'] which was actually[7] becomes [0,0,0,0,0,0,1,0,0,0,0] a 198 size array

            
            
            # categoricalcontext=to_categorical(padded_array,word_count)
            # concatarray=np.vstack(categoricalcontext)
            



            
            yield(padded_array,categoricaltarget)
            #outputs a tuple 
            #{['w2','w3',0,0],[0,0,0,0,0,1,0,0]}

            #thus for every word it outputs a tuple




        





In [183]:
generator=cbow_model(tokens,window_size,word_count)
for context,target in generator:
    context2=np.array(context)
    Z=context2.reshape(1,4)

    print(Z.shape)
    print(target.shape)
    
    print(f"context:{context2}")
    
    print(f"target:{target}")

(1, 4)
(1, 198)
context:[2 3 0 0]
target:[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0.]]
(1, 4)
(1, 198)
context:[38  3  9  0]
target:[[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.

In [184]:
#build the model description dila ahe var
#let W be the total weights finally learned
#V be the total vocabulary

model=Sequential()
#input is V
#output is W for embedding layer
#since we have window_size*2 inputs of V length each, the Embedding layer outputs W*window_size*2
model.add(Embedding(input_dim=word_count,output_dim=100,input_length=window_size*2))

#each of these W's for W*window_size*2, we mean it to give W values
#IP1  IP2  IP3  IP4
#W11  W12  W13  W14          LAMBDA LAYER SUMS ===  W11 +W12 + W13 + W14
#W21
#W31
#W41
#..
model.add(Lambda(lambda x:K.mean(x,axis=1),output_shape=(100,)))

model.add(Dense(word_count,activation="softmax"))
#Dense returns a vector of size V 
#so we went from V*window_length*2 to W to V 
#this V is the predicted target word's one hot encoding
model.compile(loss="categorical_crossentropy",optimizer="adam")

model.metrics_names


# Training loop
epochs = 10

for epoch in range(epochs):  
    cost=0   

    for context, target in cbow_model(tokens, window_size, word_count):
        # Train the model on a single batch
        X=np.array(context)

        Xnew=X.reshape(1,4)
        
        #need to reshape X to (1,4) 
        #the shape of X must be (batch_size,input_length)           
       
        cost += model.train_on_batch(x=Xnew, y=target)
    print(f"epoch no:{epoch} cost:{cost}")
        


print(model.metrics_names)


        

    
    





epoch no:0 cost:1035.5935130119324
epoch no:1 cost:927.2827455997467
epoch no:2 cost:879.8392827510834
epoch no:3 cost:866.069584608078
epoch no:4 cost:857.1325328350067
epoch no:5 cost:849.0224800109863
epoch no:6 cost:840.5724587440491
epoch no:7 cost:831.2814631462097
epoch no:8 cost:820.9262385368347
epoch no:9 cost:809.458710193634
['loss']


In [185]:
#use the learned weight to create a vector file
model.get_weights()
#returns all learned weights


[array([[-0.08611247, -0.16295907, -0.16905762, ..., -0.12273635,
          0.13962331, -0.09313374],
        [ 0.07807573, -0.09708354,  0.14009409, ..., -0.05193266,
         -0.00055228, -0.13515234],
        [-0.1820547 , -0.1310883 , -0.2034515 , ..., -0.16554199,
          0.12659295, -0.22189853],
        ...,
        [-0.00835277,  0.0174902 ,  0.02848316, ..., -0.01717081,
          0.01080253, -0.04308313],
        [-0.03801566, -0.0411983 ,  0.04060432, ..., -0.01986821,
          0.01893058, -0.04467294],
        [-0.03645645,  0.00277979,  0.02679689, ...,  0.01966368,
         -0.01598086, -0.04402192]], dtype=float32),
 array([[ 0.74240685, -0.15300004, -0.07122096, ...,  0.72182626,
          0.7477592 ,  0.695739  ],
        [ 0.7248573 ,  0.00367028, -0.1650015 , ...,  0.7525633 ,
          0.75164497,  0.5964021 ],
        [ 0.8005784 , -0.2025098 , -0.0103286 , ...,  0.604129  ,
          0.7179648 ,  0.6789633 ],
        ...,
        [ 0.67842793, -0.19438466, -0.1

In [186]:
weights=model.get_weights()[0]
#returns weights of the first word in the document
#100 sized array
print(weights)
print(weights.shape)

[[-0.08611247 -0.16295907 -0.16905762 ... -0.12273635  0.13962331
  -0.09313374]
 [ 0.07807573 -0.09708354  0.14009409 ... -0.05193266 -0.00055228
  -0.13515234]
 [-0.1820547  -0.1310883  -0.2034515  ... -0.16554199  0.12659295
  -0.22189853]
 ...
 [-0.00835277  0.0174902   0.02848316 ... -0.01717081  0.01080253
  -0.04308313]
 [-0.03801566 -0.0411983   0.04060432 ... -0.01986821  0.01893058
  -0.04467294]
 [-0.03645645  0.00277979  0.02679689 ...  0.01966368 -0.01598086
  -0.04402192]]
(198, 100)


In [187]:
vect_file=open('vectors.txt','w')
vect_file.write(f'{word_count} {100}\n')

8

In [188]:
print(vectorize.word_index.items())
#the list of words and their indexing.

for word, index in vectorize.word_index.items():
    
    word_vector = ' '.join(map(str, list(weights[index, :])))
    #''.join() needs iterable inside the bracket. and it joins all items in that iterable using whatever you specify between the ''

    # map(function,iterable) maps a function to all elements in an iterable

    #so it converts the elements inside the weight[indexth row and ,:  means all columns] which is made into a list first and sent to map function to convert it into a list of strings
    #["w1","w2","w3"]
    #next the join function joins all elements in this stringlist of weights using space
    #"w1 w2 w3"
    #we have our word vector which is a string

    vect_file.write('{} {}\n'.format(word, word_vector))
    #we write the word and word_vector for every word present inside the word_index
vect_file.close()
     

dict_items([('the', 1), ('of', 2), ('transmission', 3), ('influenza', 4), ('covid', 5), ('19', 6), ('virus', 7), ('for', 8), ('is', 9), ('to', 10), ('a', 11), ('and', 12), ('–', 13), ('between', 14), ('time', 15), ('serial', 16), ('interval', 17), ('than', 18), ('be', 19), ('5', 20), ('days', 21), ('are', 22), ('viruses', 23), ('shorter', 24), ('from', 25), ('appearance', 26), ('symptoms', 27), ('while', 28), ('3', 29), ('this', 30), ('that', 31), ('can', 32), ('in', 33), ('major', 34), ('driver', 35), ('number', 36), ('2', 37), ('speed', 38), ('an', 39), ('important', 40), ('point', 41), ('difference', 42), ('two', 43), ('has', 44), ('median', 45), ('incubation', 46), ('period', 47), ('infection', 48), ('successive', 49), ('cases', 50), ('estimated', 51), ('6', 52), ('means', 53), ('spread', 54), ('faster', 55), ('further', 56), ('first', 57), ('illness', 58), ('or', 59), ('potentially', 60), ('pre', 61), ('symptomatic', 62), ('before', 63), ('contrast', 64), ('we', 65), ('learning', 

In [190]:
cbow_output = gensim.models.KeyedVectors.load_word2vec_format('vectors.txt',binary=False,limit=100,unicode_errors='ignore')

In [195]:
cbow_output.most_similar(positive=['symptomatic'])

[('days', 0.18036538362503052),
 ('prior', 0.17420361936092377),
 ('an', 0.16353300213813782),
 ('speed', 0.15734535455703735),
 ('very', 0.15580981969833374),
 ('5', 0.1396934688091278),
 ('incubation', 0.1389920711517334),
 ('further', 0.1304580569267273),
 ('of', 0.12922650575637817),
 ('estimated', 0.12896320223808289)]